# Data Processing and Ingestion

## Introduction

This notebook handles various input data types, transforming them into a document format, and then incorporates them into a vector database for subsequent processing. The indexed documents are further used in the next notebook to create QnA function and deploy it on Watsonx.AI. 

The initial section of the notebook focuses onfocuses on handling any type of data contained within a directory or a zip file. The process involves extracting content from different file types, segmenting the data, converting it into a document format, and finally indexing the content into an Elasticsearch vector database.

The ingestion process leverages vector embeddings to optimize data storage and retrieval within Elasticsearch, ensuring efficiency and effectiveness. In the latter part of the notebook, the following actions are performed:

- Establishing a connection to Elasticsearch and loading input data from processed documents.
- Splitting and indexing the documents into Elasticsearch.
- Utilizing an Elastic Learned Sparse EncodeR model (ELSER) and langchain to search and retrieve related documents based on specific questions. 

**Note**: Langchain is a framework designed to simplify the creation of applications using large language models. 

## Contents

This notebook contains the following parts:
- [Setup](#setup)
- [Input file or URL name](#input)
- [Extract Data](#extract)
- [Get Model status](#model)
- [Index Documents using ibm_watsonx_ai's vectorstore](#insert_documents)
- [Q&A using Vectorstore and query templates](#QnATest)




Below cell installs specific libraries required to run this notebook

<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Download and import mandatory libraries and dependencies

In [ ]:
!pip install unstructured==0.7.12 | tail -n 1
!pip install elasticsearch | tail -n 1
!pip install langchain | tail -n 1
!pip install tiktoken | tail -n 1
!pip install langchain-community
!pip install pypdf | tail -n 1
!pip install langchain_elasticsearch | tail -n 1
!pip install ibm_watsonx_ai==1.0.4 | tail -n 1

The below cell imports  libraries required to run this notebook and imports the `rag_helper_functions.py` script stored in the project asset. The script is a dependency to execute the notebooks. 

In [ ]:
from elasticsearch import Elasticsearch, helpers

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from ibm_watsonx_ai import APIClient
import hashlib
from bs4 import BeautifulSoup
import multiprocessing
import json
import os
from ibm_watsonx_ai.foundation_models.extensions.rag import VectorStore
import asyncio

import zipfile
from threading import Thread
import shutil 
import warnings
from tqdm import tqdm
import time
warnings.filterwarnings("ignore")

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()
   






<a id="parameterimport"></a>
### Import Parameter Sets
Once the rag_helper_functions is imported, the `rag_helper_functions` methods are used to retrieve the parameter set from the project. <br>
* **`get_parameters()`** - Retrieves the parameters stored in the project using the bearer token.


In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]
parameters={}
for parameter_set in parameter_sets:
    parameter_vals= wslib.assets.get_asset(parameter_set, "parameter_set", raw=True)
    if 'entity' in parameter_vals and 'parameter_set' in parameter_vals['entity'] and 'parameters' in parameter_vals['entity']['parameter_set']:
        parameter_vals = parameter_vals['entity']
    params = {param['name']: param['value'] for param in parameter_vals['parameter_set']['parameters']}
    parameters.update(params)    

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = {"apikey": ibm_api_key, "url": WML_SERVICE_URL}
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {"token": token,"instance_id" : "openshift","url": hostname}

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)


### Enter a file name for loading, processing and indexing. 

Enter one of the following options:

- The name of a zip file containing .pdf/.docx/.pptx/.html files that exists in the project as a data asset. e.g. ibm-docs-SSYOK8.zip
- The name of a .pdf/.docx/.pptx/.html file that exists in the project as a data asset. 
- Watsonx.ai html directory URL. i.e. https://www.ibm.com/docs/api/v1/toc/SSYOK8 

In [ ]:
asset_li=wslib.assets.list_assets("data_asset")
if "Env_Asset" in os.environ:
    file_id=os.environ["Env_Asset"].split("/")[-1]
    input_filename = next((asset['name'] for asset in asset_li if asset['asset_id'] == file_id), None) or "ibm-docs-SSYOK8.zip"
else:
    input_filename =input("Enter a name for the data asset(.zip/.pdf/.docx/.pptx/.html) or enter IBM Docs html directory:") or "ibm-docs-SSYOK8.zip"
    
    
if any(input_filename.lower().endswith(ext) for ext in ['.zip', '.pdf', '.docx', '.pptx', '.html']):
    wslib.download_file(input_filename)
elif input_filename =="https://www.ibm.com/docs/api/v1/toc/SSYOK8":
    scriptname = 'download_ibm_docs.py'
    wslib.download_file(scriptname)

    import download_ibm_docs
    await download_ibm_docs.extract_content_and_save_async(input_filename)
    dir_name = input_filename.split("/")[-1]
    download_ibm_docs.zip_folder(dir_name,dir_name)
    input_filename = dir_name+".zip"
    

if the format of the file is `.zip` then below cell extracts the files in the zip into a directory.

In [ ]:

directory ="watson-docs"


if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory)
if ".zip" in input_filename:
    with zipfile.ZipFile(input_filename, 'r') as zip_ref:
        zip_ref.extractall(path=directory)

    print("Extraction completed!")
else:
    shutil.move(input_filename, directory)
    print("File copied")


### Load all the Documents 

The following cell generates a `DirectoryLoader` instance from the langchain library, configured to read files with extensions such as .html, .pdf, .pptx, and .docx. The loader traverses the specified folder, gathers all the documents, and appends them to a list. 

In the subsequent cell, these documents are processed to be inserted into a vector database. This involves splitting the documents using langchain's `RecursiveCharacterTextSplitter` and incorporating both content and metadata into the documents. The term "recursive" suggests that this division process happens in multiple stages or levels, breaking down the text into increasingly smaller segments.

In [ ]:

loaders = [
    DirectoryLoader(directory, glob="**/*.html",show_progress=True),
    DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
    DirectoryLoader(directory, glob="**/*.pptx",show_progress=True),
    DirectoryLoader(directory, glob="**/*.docx",show_progress=True)
]


documents=[]
for loader in loaders:
    data =loader.load()
    documents.extend(data)

The cell below prepares documents for insertion into a vector database, specifically Elasticsearch in our context. It handles HTML files where document URLs are stored within the `canonical` tag. The cell reads the `canonical` tag and incorporates it into the HTML files metadata.

In [ ]:
content=[]
metadata = []
for doc in documents:
    
    document_url = ""
    document_title = doc.metadata["title"] if "title" in doc.metadata else doc.metadata['source'].split("/")[-1].split(".")[0]

    if ".html" in doc.metadata['source']:
        with open(doc.metadata['source'], 'r', encoding='utf-8') as html_file:
            html_content = html_file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        canonical_tag = soup.find('link', {'rel': 'canonical'})
        title_tag = soup.find('title')

        if canonical_tag:
            document_url = canonical_tag.get('href')
        
        if title_tag:
            document_title=title_tag.get_text()
    
    metadata.append({
            "title":document_title ,
            "source": doc.metadata['source'],
            "document_url":document_url,
            "page_number":doc.metadata['page'] if 'page' in doc.metadata else ''
            
        })
    
    content.append("Document Title: " + document_title+"\n"+"Document Content: "+doc.page_content)
    


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=parameters['ingestion_chunk_size'],
    chunk_overlap=parameters['ingestion_chunk_overlap'],
    disallowed_special=()
)

split_docs = text_splitter.create_documents(content, metadatas=metadata)
print(f"{len(documents)} Documents are split into {len(split_docs)} documents with a chunk size",parameters["ingestion_chunk_size"])

id_list=[]
for doc in split_docs:
    id_list.append(hashlib.sha256(doc.page_content.encode()).hexdigest())  
print(len(id_list) - len(set(id_list)),"duplicate documents found.")
print(len(set(id_list)), "Documents will be indexed into vector store")

<a id="ESConnect"></a>
### Connecting to Elasticsearch Instances
Elasticsearch can be connected in various ways, depending on your environment and requirements. Below are detailed instructions and necessary parameters for establishing a connection using the Python client. For a more comprehensive guide, refer to the [official documentation on Elasticsearch connection using the Python client.](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html)

#### Connecting using Project Connection Asset (default)
The notebook, by default, will look for a connection asset in the project named `elasticsearch_connect`.  You can set this up by following the instructions in the project readme. 
Use this method if you are connecting to an Elasticsearch environment using a connection URL, username and password



In [ ]:
connection_name=parameters["elastic_search_connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    es_connection = wslib.get_connection(connection_name)
    
    print("Successfully retrieved the connection details")
    es_client=rag_helper_functions.create_elastic_client(es_connection)
else:
    es_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.")
    

<a id="model"></a>
### Check if ELSER model is deployed
Get the status of the current Elasticsearch model (e.g. ELSER 1/ELSER 2). For this ensure that you have the model downloaded and deployed on Elasticsearch. This can be done under **Machine Learning >> Trained Models** section on Elasticsearch.

In [ ]:
try:
    status = es_client.ml.get_trained_models_stats(model_id=parameters['elastic_search_model_id'])
    if status["trained_model_stats"][0]["deployment_stats"]["state"] != "started":
        raise Exception("Model is downloaded but not ready to be deployed.")
    print(parameters['elastic_search_model_id'], "is ready and deployed. This will be used to index the documents.")
    
except Exception as e:
    raise Exception("Error: Model not found or yet to be deployed, kindly check and try again.") from e

<a id="insert_documents"></a>
### Inserting Documents using Langchains Elastic Search Vector Store

Below code utilizes `ibm_watsonx_ai`s Elasticsearch vector store extension to store documents. <br>
* Firstly, it creates a connection to Elasticsearch and defines how documents will be retrieved later. <br>
* Then, it defines a function to add these documents to the vector store. This function takes the documents and additional parameters for efficient processing, such as splitting the documents into smaller chunks and setting a timeout for requests. <br>


Overall, this code efficiently adds a list of documents to a vector store hosted in Elasticsearch, making them searchable. <br>


In [ ]:
vector_store = VectorStore(
    client=client,
    connection_id=es_connection['.']['asset_id'],
    data_source_type='elasticsearch',
    params={ 
         "index_name":parameters["elastic_search_index_name"],
         "model_id": parameters['elastic_search_model_id'],
         "vector_query_field":parameters['vector_query_field']
         
     }
    
)

In [ ]:
documents=asyncio.create_task(
    vector_store.add_documents_async(
    split_docs, 
    bulk_kwargs={
                "chunk_size": parameters['elastic_search_index_chunk_size'],
                "request_timeout": 600
            })
)

#documents=vector_store.add_documents(split_docs, bulk_kwargs={ "chunk_size": parameters['elastic_search_index_chunk_size'],"request_timeout": 600}) #Synchronous call

## Q&A on the Elastic Search Index
<a id="QnATest"></a>
### 1. Using the Vector Store

The following sections of the notebook are designed to test a sample Question and Answer (QnA) interaction using the ELSER model, assuming it is utilized. The subsequent cell in the notebook executes this test and provides a response that includes several key pieces of information. 

**Note**: if you run this cell immediately after kicking off the indexing process, the results may not always be accurate as the indexing process may not have completed. You can goto **Index Management** section on your elastic search instance user interface to see the progress of the documents being indexed. 

In [ ]:
question ="how to perform sql optimization?"

Uncomment the section in the below cell and include the filter section in case you would like to query based on conditions.

In [ ]:
try:

    result = vector_store.search(question, k=parameters['rag_es_top_n_results'], include_scores=True, verbose=True)#, filter=[{"match": {"metadata.title": {"query": "sql_overview", "fuzziness": "AUTO"}}}])

except Exception as e:
    print("An error occurred while querying elastic search, please retry later:", e)



### 2. Using Elastic Search Query Templates

The following section of the notebook is designed to test a sample Question and Answer (QnA) interaction using sample template of ELSER model, assuming it is utilized.
This response comprises of:

* `Relevance Score`: A numerical value indicating the relevance or confidence level of the answer provided by the model.
* `Title`: The title of the document from which the answer is derived.
* `Document ID`: A unique identifier for the document within the database or index.
* `Document URL`: The location where the original document can be accessed or referenced.
* `Document Content`: The actual content or text from the document that is relevant to the queried question.
* `Source`: The source of the content or text for the relevant document queried. 
* `Page Number`: Page Number of the document (if applicable).

This setup allows for a practical demonstration of the model's capabilities in retrieving and presenting information in response to a specific query. 
There are 2 ways to perform this step, depending on the `elastic_search_template_file` parameter provided in the parameter set by the user. 
1. **ELSER**: An ELSER exclusive search query is invoked.
2. **ELSER + BM25**: A hybrid search query that is a combination of a tradition vector search and ELSER is invoked.


In [ ]:
wslib.download_file(parameters['elastic_search_template_file'])
with open(parameters['elastic_search_template_file']) as f:
    es_query = json.load(f)

print("updating tokens and query fields")

if "BM25" in parameters['elastic_search_template_file']:
    es_query['bool']['should'][0]['match']['description']['query'] = question
    es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
    es_query['bool']['should'][1]['text_expansion']['vector.tokens']['model_text'] = question
    print("ELSER + BM25 Lexical search query : ")
else:
    es_query['text_expansion']['vector.tokens']['model_id'] = parameters['elastic_search_model_id']
    es_query['text_expansion']['vector.tokens']['model_text'] = question
    print("ELSER search query : ")
    
print(es_query)


try:
    response = es_client.search(
        index=parameters["elastic_search_index_name"], 
        size=parameters['rag_es_top_n_results'],
        query=es_query
    )
    print("\nResponse:")
    for hit in response['hits']['hits']:

        score = hit['_score']
        title = hit['_source']['metadata']['title']
        page_content=hit['_source']['text']
        source = hit['_source']['metadata']['source']
        url = hit['_source']['metadata']['document_url']
        page_number = hit['_source']['metadata']['page_number']


        print(f"\nRelevance Score  : {score}\nTitle            : {title}\nSource     : {source}\nDocument Content : {page_content}\nDocument URL : {url}\nPage Number : {page_number}")

except Exception as e:
        print("An error occurred while querying elastic search, please retry after sometime:", e)
    



Below cell is an optional step for continuously monitoring the documents count in the Elastic Search Instance.  **You can proceed to `Create and Deploy Q&A Python Function` notebook to create and deploy the RAG python function without waiting for the next cell to complete.**

In [ ]:
def monitor_status(total_doc_count):
    current_doc_count =0 

    with tqdm(total=total_doc_count, desc="Indexing Progress") as progressbar:
        while current_doc_count < total_doc_count:
            time.sleep(30)
            try:
                es_client.indices.refresh(index=parameters["elastic_search_index_name"])
                current_doc_count = es_client.count(index=parameters["elastic_search_index_name"])['count']
            except Exception as e:
                print("An error occurred while checking indexing status from elastic search, please rerun the cell:", e)            
            progressbar.update(current_doc_count-progressbar.n)


monitor_status(len(set(id_list)))                                         # synchronous (wait for indexing completion)
#multiprocessing.Process(target=monitor_status, args=(len(set(id_list)),)).start()  # asynchronous (continue notebook while indexing) 




**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**
